In [302]:
import pandas as pd
import zipfile
#import kaggle

In [303]:
#!kaggle datasets download -d aijobs/global-salaries-in-ai-ml-data-science

In [304]:
zipfile_name = 'global-salaries-in-ai-ml-data-science.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [305]:
df_salary = pd.read_csv('salaries.csv')
df_salary.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,MI,FT,Data Engineer,138000,USD,138000,US,0,US,M
1,2024,MI,FT,Data Engineer,86000,USD,86000,US,0,US,M
2,2024,EN,FT,Data Analyst,179000,USD,179000,US,0,US,M
3,2024,EN,FT,Data Analyst,112000,USD,112000,US,0,US,M
4,2024,MI,FT,Business Intelligence Developer,92000,USD,92000,US,0,US,M


In [306]:
print ('Rows x Columns: ', df_salary.shape, '\n')
print ('--------')
print ("Columns and their types: ")
df_salary.dtypes

Rows x Columns:  (10670, 11) 

--------
Columns and their types: 


work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [307]:
print ('Year range: ', df_salary.work_year.min(),' - ', df_salary.work_year.max())
print ('Experience Levels: ', df_salary['experience_level'].unique())
print ('Employment Type: ', df_salary['employment_type'].unique())
print ('Company Size: ', df_salary['company_size'].unique())

Year range:  2020  -  2024
Experience Levels:  ['MI' 'EN' 'SE' 'EX']
Employment Type:  ['FT' 'PT' 'CT' 'FL']
Company Size:  ['M' 'L' 'S']


In [308]:
#count null values per column
df_salary.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

#### Statistics

In [309]:
df_salary.describe(include='object')

,experience_level,employment_type,job_title,salary_currency,employee_residence,company_location,company_size
count,10670,10670,10670,10670,10670,10670,10670
unique,4,4,132,22,86,75,3
top,SE,FT,Data Engineer,USD,US,US,M
freq,7439,10624,2438,9771,9221,9270,9662


In [310]:
print ('There are', len(df_salary.job_title.value_counts()), 'different job titles.\n')
print ('The 10 most frequency jobs are:')
df_salary.job_title.value_counts().head(10)

There are 132 different job titles.

The 10 most frequency jobs are:


job_title
Data Engineer                     2438
Data Scientist                    2222
Data Analyst                      1600
Machine Learning Engineer         1118
Research Scientist                 317
Applied Scientist                  298
Analytics Engineer                 284
Data Architect                     253
Research Engineer                  175
Business Intelligence Engineer     164
Name: count, dtype: int64

#### Combine jobs

In [311]:
# There are jobs that are the same job but is written the different way, like `BI analyst` and `Bussiness Intelligence analyst`

df_salary.replace("Business Intelligence Analyst", "BI Analyst", inplace=True)
df_salary.replace("Business Intelligence Developer", "BI Developer", inplace=True)
df_salary.replace("Business Intelligence Engineer", "BI Engineer", inplace=True)
df_salary.replace("ML Engineer", "Machine Learning Engineer", inplace=True)
df_salary.replace("Data Science", "Data Scientist", inplace=True)
df_salary.replace("Data Analytics Engineer", "Data Engineer", inplace=True)
df_salary.replace("Staff Data Analyst", "Data Analyst", inplace=True)
df_salary.replace("Data Science Practitioner", "Data Scientist", inplace=True)
df_salary.replace("AI Research Engineer", "AI Engineer", inplace=True)

print ('There are', len(df_salary.job_title.value_counts()), 'different job titles.\n')
print ('The 10 most frequency jobs are:')
df_salary.job_title.value_counts().head(10)

There are 124 different job titles.

The 10 most frequency jobs are:


job_title
Data Engineer                2443
Data Scientist               2274
Data Analyst                 1603
Machine Learning Engineer    1240
Research Scientist            317
Applied Scientist             298
Analytics Engineer            284
Data Architect                253
Research Engineer             175
BI Engineer                   164
Name: count, dtype: int64

#### Replace _experience_level_ values

In [312]:
df_salary.experience_level.replace('EN', 'Entry Level / Junior', inplace=True)
df_salary.experience_level.replace('MI', 'Intermediate', inplace=True)
df_salary.experience_level.replace('SE', 'Senior / Expert', inplace=True)
df_salary.experience_level.replace('EX', 'Executive / Director', inplace=True)

#### Remove jobs infrequently

In [313]:
# keep rows with `job_title` frequency is at least 10
job_title_counts = df_salary['job_title'].value_counts()

job_titles_to_keep = job_title_counts[job_title_counts >= 10].index

filtered_df = df_salary[df_salary['job_title'].isin(job_titles_to_keep)]
to_remove = df_salary[~df_salary['job_title'].isin(job_titles_to_keep)]


print('Were removed', df_salary.shape[0]-filtered_df.shape[0], 'rows.')

Were removed 193 rows.


#### Average salary

In [314]:
# Salary per job title and experience
average_salary = filtered_df.groupby(['job_title', 'experience_level', 'work_year'])['salary_in_usd'].mean().reset_index()
average_salary.rename(columns={'salary_in_usd': 'average_salary_in_usd'}, inplace=True)
average_salary


,job_title,experience_level,work_year,average_salary_in_usd
0,AI Architect,Executive / Director,2023,215936.000000
1,AI Architect,Senior / Expert,2023,253454.545455
2,AI Architect,Senior / Expert,2024,229662.500000
3,AI Developer,Entry Level / Junior,2023,110119.500000
4,AI Developer,Intermediate,2023,138294.333333
...,...,...,...,...
345,Research Scientist,Intermediate,2024,78250.000000
346,Research Scientist,Senior / Expert,2021,68956.666667
347,Research Scientist,Senior / Expert,2022,186082.000000
348,Research Scientist,Senior / Expert,2023,195332.030000


In [315]:
#filtered_df.groupby(['job_title'])['experience_level'].value_counts()

#### Add columns: country names

In [316]:
#import file with country names

country_codes = pd.read_csv('ISO-3166-country-code.csv')
country_codes.head()

,name,alpha-2,country-code
0,Afghanistan,AF,4
1,Åland Islands,AX,248
2,Albania,AL,8
3,Algeria,DZ,12
4,American Samoa,AS,16


In [317]:
#merge dataframes filtered_df and country_codes to add 'country_residence column
filtered_df = pd.merge(filtered_df, country_codes, how='inner', left_on='employee_residence', right_on='alpha-2')
filtered_df.drop(columns=['alpha-2', 'country-code'], inplace=True)

filtered_df.rename(columns={'name': 'country_residence'}, inplace=True)
filtered_df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,country_residence
0,2024,Intermediate,FT,Data Engineer,138000,USD,138000,US,0,US,M,United States of America
1,2024,Intermediate,FT,Data Engineer,86000,USD,86000,US,0,US,M,United States of America
2,2024,Entry Level / Junior,FT,Data Analyst,179000,USD,179000,US,0,US,M,United States of America
3,2024,Entry Level / Junior,FT,Data Analyst,112000,USD,112000,US,0,US,M,United States of America
4,2024,Intermediate,FT,BI Developer,92000,USD,92000,US,0,US,M,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...
10472,2021,Entry Level / Junior,PT,Computer Vision Engineer,180000,DKK,28609,DK,50,DK,S,Denmark
10473,2020,Entry Level / Junior,FT,AI Scientist,300000,DKK,45896,DK,50,DK,S,Denmark
10474,2021,Intermediate,FT,Big Data Engineer,18000,USD,18000,MD,0,MD,S,"Moldova, Republic of"
10475,2021,Entry Level / Junior,FT,Business Data Analyst,50000,EUR,59102,LU,100,LU,L,Luxembourg


In [318]:
#merge dataframes filtered_df and country_codes to add 'country_company' column

filtered_df = pd.merge(filtered_df, country_codes, how='inner', left_on='company_location', right_on='alpha-2')
filtered_df.drop(columns=['alpha-2', 'country-code'], inplace=True)

filtered_df.rename(columns={'name': 'country_company'}, inplace=True)
filtered_df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,country_residence,country_company
0,2024,Intermediate,FT,Data Engineer,138000,USD,138000,US,0,US,M,United States of America,United States of America
1,2024,Intermediate,FT,Data Engineer,86000,USD,86000,US,0,US,M,United States of America,United States of America
2,2024,Entry Level / Junior,FT,Data Analyst,179000,USD,179000,US,0,US,M,United States of America,United States of America
3,2024,Entry Level / Junior,FT,Data Analyst,112000,USD,112000,US,0,US,M,United States of America,United States of America
4,2024,Intermediate,FT,BI Developer,92000,USD,92000,US,0,US,M,United States of America,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10472,2021,Entry Level / Junior,FT,Research Scientist,100000,USD,100000,JE,0,CN,L,Jersey,China
10473,2021,Entry Level / Junior,PT,Computer Vision Engineer,180000,DKK,28609,DK,50,DK,S,Denmark,Denmark
10474,2020,Entry Level / Junior,FT,AI Scientist,300000,DKK,45896,DK,50,DK,S,Denmark,Denmark
10475,2021,Intermediate,FT,Big Data Engineer,18000,USD,18000,MD,0,MD,S,"Moldova, Republic of","Moldova, Republic of"


In [319]:
#export to csv to continue working in Tableau
filtered_df.to_csv('cleaned_data_salary.csv')